In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt 
import netCDF4 as nc
from netCDF4 import Dataset
import geopandas as gpd
import pandas as pd
import random
import xarray as xr

In [3]:
# Import all standardized features 

sinmod = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/SINMOD_bottom_features_emod_grid_standardized.nc')

emod = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/EMOD_standardized.nc')


In [4]:
#current aspect angle
sinmod_bottom = xr.open_dataset('/cluster/projects/itk-SINMOD/coral-mapping/data/processed_data/features/ready-for-training/SINMOD_bottom_features.nc')
current_aspect_angle = sinmod_bottom['current_aspect_angle']

In [6]:
# EMOD first
emod_df = emod.to_dataframe().reset_index()
print(emod_df.columns)

Index(['x', 'y', 'aspect_cos', 'aspect_sin', 'bathymetry', 'broad_BPI',
       'fine_BPI', 'log_ruggedness', 'slope'],
      dtype='object')


In [4]:
print(list(sinmod.variables))

['stat', 'x', 'y', 'spatial_ref', 'bottom_temperature_sundahl_features', 'bottom_salinity_features', 'bottom_current_features', 'bottom_statistical_northness_features', 'bottom_statistical_eastness_features']


In [7]:
sinmod_df = sinmod[['x', 'y', 'bottom_temperature_sundahl_features', 'bottom_salinity_features',
                                  'bottom_current_features', 'bottom_statistical_northness_features',
                                  'bottom_statistical_eastness_features']].to_dataframe().reset_index()


In [14]:
print(aspect_bathymetry.shape)


print(aspect_current.shape)

(80511075,)
(11447, 11162)


In [16]:
print(emod_df['fine_BPI'].shape)

(80511075,)


In [17]:
print(emod_standardized['bathymetry'].shape)

NameError: name 'emod_standardized' is not defined

In [11]:
# Creating the current - aspect angle feature. Absolute difference between the current direction and the depth aspect direction

aspect_bathymetry = emod_df['aspect_cos']
aspect_current = sinmod['bottom_statistical_northness_features'].sel(stat='mean')

sinmod_df['current_aspect_angle'] = abs(np.arccos(aspect_current) - np.arccos(aspect_bathymetry)) * 180 / np.pi

/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/xarray/core/computation.py:824: RuntimeWarning: invalid value encountered in arccos
  result_data = func(*input_data)
/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)


ValueError: operands could not be broadcast together with shapes (11447,11162) (80511075,) 

In [9]:
print(len(current_aspect_angle))

9671


In [8]:
sinmod_df['current_aspect_angle'] = current_aspect_angle

ValueError: Length of values (9671) does not match length of index (638857070)

In [5]:
#Create mean, min and max features for desired variables
sinmod_df['temperature_sundahl_mean'] = sinmod_df['bottom_temperature_sundahl_features'].mean()
sinmod_df['temperature_sundahl_10th_percentile'] = sinmod_df['bottom_temperature_sundahl_features'].quantile(0.1)
sinmod_df['temperature_sundahl_90th_percentile'] = sinmod_df['bottom_temperature_sundahl_features'].quantile(0.9)

In [6]:
sinmod_df['salinity_mean'] = sinmod_df['bottom_salinity_features'].mean()
sinmod_df['salinity_10th_percentile'] = sinmod_df['bottom_salinity_features'].quantile(0.1)
sinmod_df['salinity_90th_percentile'] = sinmod_df['bottom_salinity_features'].quantile(0.9)

In [7]:
sinmod_df['current_mean'] = sinmod_df['bottom_current_features'].mean()
sinmod_df['current_10th_percentile'] = sinmod_df['bottom_current_features'].quantile(0.1)
sinmod_df['current_90th_percentile'] = sinmod_df['bottom_current_features'].quantile(0.9)

In [14]:
print(sinmod_df.columns)

Index(['x', 'y', 'stat', 'bottom_temperature_sundahl_features',
       'bottom_salinity_features', 'bottom_current_features',
       'bottom_statistical_northness_features',
       'bottom_statistical_eastness_features', 'temperature_sundahl_mean',
       'temperature_sundahl_10th_percentile',
       'temperature_sundahl_90th_percentile', 'salinity_mean',
       'salinity_10th_percentile', 'salinity_90th_percentile', 'current_mean',
       'current_10th_percentile', 'current_90th_percentile'],
      dtype='object')


In [8]:
sinmod_df = sinmod_df.drop(['stat', 'bottom_temperature_sundahl_features','bottom_salinity_features', 'bottom_current_features' ], axis=1)

In [9]:
print(sinmod_df.columns)

Index(['x', 'y', 'bottom_statistical_northness_features',
       'bottom_statistical_eastness_features', 'temperature_sundahl_mean',
       'temperature_sundahl_10th_percentile',
       'temperature_sundahl_90th_percentile', 'salinity_mean',
       'salinity_10th_percentile', 'salinity_90th_percentile', 'current_mean',
       'current_10th_percentile', 'current_90th_percentile'],
      dtype='object')


In [ ]:
total_df = sinmod_df.merge(emod_df, on=['x', 'y'])

In [30]:
print(total_df.columns)

Index(['x', 'y', 'stat', 'bottom_temperature_sundahl_features',
       'bottom_salinity_features', 'bottom_current_features',
       'bottom_statistical_northness_features',
       'bottom_statistical_eastness_features', 'current_aspect_angle',
       'bottom_temperature_sundahl_features_mean',
       'bottom_salinity_features_mean', 'bottom_current_features_mean',
       'bottom_temperature_sundahl_features_10th_percentile',
       'bottom_salinity_features_10th_percentile',
       'bottom_temperature_sundahl_features_90th_percentile',
       'bottom_salinity_features_90th_percentile',
       'bottom_current_features_90th_percentile', 'aspect_cos', 'aspect_sin',
       'bathymetry', 'broad_BPI', 'fine_BPI', 'log_ruggedness', 'slope'],
      dtype='object')


In [ ]:
#sinmod_df = sinmod_df.dropna()
#emod_df = emod_df.dropna()


In [31]:
total_df.to_parquet('/cluster/home/maikents/features_midnor_2019/ready_for_training/total_df.parquet')

In [36]:
#To open later: total_df = pd.read_parquet('/cluster/home/maikents/features_midnor_2019/ready_for_training/total_df.parquet')